In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as apps
import pandas as pd
import kagglehub
from pathlib import Path
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split

2025-04-07 23:36:50.224866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744065410.244064   70721 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744065410.249672   70721 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744065410.266348   70721 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744065410.266393   70721 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744065410.266395   70721 computation_placer.cc:177] computation placer alr

In [2]:
SEED = 42
BATCH_SIZE = 16
PATIENCE = 5

saved_models_dir = Path("../saved_models")
saved_models_dir.mkdir(parents=True, exist_ok=True)
histories_dir = Path("../histories")
histories_dir.mkdir(parents=True, exist_ok=True)

In [3]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def set_memory_limit(memory_limit):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
        )

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

set_memory_limit(4096)
#set_memory_growth()
keras.mixed_precision.set_global_policy("mixed_float16")

pd.set_option('display.max_columns', None)

1 Physical GPU, 1 Logical GPUs


I0000 00:00:1744065413.532953   70721 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


# Create Datasets

In [4]:
# Download latest version of data
image_dir = kagglehub.dataset_download("bloox2/fieldplant")
image_dir = Path(image_dir) / "train"
print("Path to dataset files:", image_dir)

Path to dataset files: /home/ruairi/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train


# Create Models

In [5]:
def get_model_and_preprocessing(app_name):
    app = getattr(keras.applications, app_name)
    model_name = dir(app)[0]
    model = getattr(app, model_name)
    input_shape = model().input_shape[1:]
    model = model(include_top=False, input_shape=input_shape)
    model.trainable = False
    preprocessing = getattr(app, "preprocess_input")
    return model, preprocessing

In [6]:
def get_pre_classifier_layers(model_name):
    if "vgg" in model_name:
        return keras.Sequential([
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(1024, activation="relu"),
            keras.layers.Dense(1024, activation="relu") ])
    else:
        return keras.Sequential([keras.layers.GlobalAveragePooling2D()])

In [7]:
def build_model(app_name, activation, num_classes):
    model, preprocessing = get_model_and_preprocessing(app_name)
    pre_classifier_layers = get_pre_classifier_layers(model.name)

    inputs = keras.Input(shape=model.input_shape[1:])
    x = preprocessing(inputs)
    x = model(x, training=False)
    x = pre_classifier_layers(x)
    outputs = keras.layers.Dense(num_classes, activation=activation, name="classifier_layer")(x)

    model_name = model.name
    model = keras.Model(inputs, outputs, name=model_name)

    return model

In [8]:
def get_hyperparameters(methodology):
    multilabel = methodology == "multilabel"
    
    methodologies = ["multiclass", "multilabel"]
    losses = ["categorical_crossentropy", "binary_crossentropy"]
    activation = ["softmax", "sigmoid"]
    metrics = ["categorical_accuracy", "binary_accuracy"]

    idx = methodologies.index(methodology)

    metrics = [metrics[idx]]
    f1_score_weighted = keras.metrics.F1Score(average="weighted", threshold=0.5 if multilabel else None, name="f1_score_weighted", dtype=None)
    f1_score_per_class = keras.metrics.F1Score(average=None, threshold=0.5 if multilabel else None, name="f1_score_per_class", dtype=None)
    metrics.append(f1_score_weighted)
    metrics.append(f1_score_per_class)
        
    hyperparams = [losses[idx], activation[idx], metrics]
    
    return hyperparams

In [9]:
def print_model_info(model):
    compile_config = model._compile_config.config
    optimizer = compile_config['optimizer'].get_config()
    classifier_activation = model.get_layer(name="classifier_layer").activation.__name__

    print("Model name:", model.name)
    print("Input shape:", model.input_shape)
    print("Optimizer name:", optimizer['name'], "learning_rate:", np.round(optimizer['learning_rate'], 6))
    print("Loss:", compile_config['loss'])
    print("Metrics:")
    for metric in compile_config['metrics']:
        print(metric if isinstance(metric, str) else metric.get_config())
    print("Classifier layer activation function:", classifier_activation)
    print()

In [10]:
def get_callbacks(model_name, fine_tuning=False):
    cbs = [
        keras.callbacks.EarlyStopping(patience=PATIENCE, restore_best_weights=True, baseline=None, verbose=1),
        keras.callbacks.ModelCheckpoint(filepath=f"{saved_models_dir}/{model_name}.keras", save_best_only=True, monitor="val_loss", verbose=1, initial_value_threshold=None)]
    return cbs


# GET DATASETS

In [11]:
def get_dataframe(filtered=False, sample=False):
    filename = "filtered" if filtered else "unfiltered"
    all_csv_files = list(Path("../data").glob("*"))
    csv_file = [csv for csv in all_csv_files if filename in csv.name][0]
    df = pd.read_csv(csv_file)
    print()

    if sample:
        print("Using sampled DF")
        df = df.sample(frac=0.2)
        df = df.loc[(df!=0).any(axis=1)]
    print("df shape:", df.shape)
    return df   

In [12]:
def get_train_test_splits(df, filtered=False, test_size=0.2):
    col_names = list(df.columns)
    split_fn = get_stratified_splits if filtered else get_nonstratified_splits
    (X_train, X_test, X_val, y_train, y_test, y_val) = split_fn(df, test_size=test_size)
    train_df = pd.merge(X_train, y_train, left_index=True, right_index=True)
    test_df = pd.merge(X_test, y_test, left_index=True, right_index=True)
    val_df = pd.merge(X_val, y_val, left_index=True, right_index=True)
    
    train_df.columns = col_names
    test_df.columns = col_names
    val_df.columns = col_names

    return train_df, test_df, val_df
    
def get_stratified_splits(df, test_size=0.2):
    columns = list(df.columns)
    X = df.filename.to_frame().to_numpy()
    y = df.drop(columns=["filename"]).to_numpy()

    X_train, y_train, X_test_val, y_test_val = iterative_train_test_split(X, y, test_size=0.2)
    X_test, y_test, X_val, y_val = iterative_train_test_split(X_test_val, y_test_val, test_size=0.5)
    datasets = (X_train, X_test, X_val, y_train, y_test, y_val)
    datasets = [pd.DataFrame(dataset) for dataset in datasets]
    return tuple(datasets)

def get_nonstratified_splits(df, test_size=0.2):
    X = df.filename
    y = df.drop(columns=["filename"])
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=SEED)

    return (X_train, X_test, X_val, y_train, y_test, y_val)

In [13]:
def decode_img(filename, img_size):
    filepath = str(image_dir) + "/" + filename
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img
    
def process_dataset(filename, labels, img_size):
    img = decode_img(filename, img_size=img_size)
    return img, labels

def configure_datasets_for_performance(datasets, shuffle=False, batch_size=BATCH_SIZE):
    configured_datasets = []
    for dataset in datasets:
        if shuffle:
            dataset = dataset.shuffle(buffer_size=dataset.cardinality(), reshuffle_each_iteration=True)
        dataset = dataset.batch(batch_size=batch_size, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.cache()
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        configured_datasets.append(dataset)
    return tuple(configured_datasets)

In [14]:
def datasets_from_dataframes(img_size, splits=None):
    datasets = []
    for split in splits:
        img = split.filename
        labels = split.drop(columns=["filename"])
        dataset = tf.data.Dataset.from_tensor_slices((img, labels))
        dataset = dataset.map(lambda x,y: process_dataset(x,y, img_size))
        datasets.append(dataset)
    return tuple(datasets)

In [15]:
def get_datasets(img_size, df, test_size=0.2):
    splits = get_train_test_splits(df, test_size=test_size)
    datasets = datasets_from_dataframes(img_size, splits=splits)
    for dataset in zip(["train", "val", "test"], datasets):
        print(dataset[0],"size:", len(dataset[1]))
    return datasets

# CREATE MODELS

In [16]:
def run_all(epochs=5, sample=True):
    i = 0
    num_models = 0

#    app_names = ["mobilenet_v2", "vgg16", "inception_v3", "inception_resnet_v2"]
    # app_names = ["mobilenet_v2", "vgg16"]
    app_names = ["vgg16"]
    
    methodologies = ["multiclass", "multilabel"]
    filter_options = [False, True]

    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
            for app_name in app_names:
                num_models += 1

    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
                
            loss, activation, metrics = get_hyperparameters(methodology)
    
            df = get_dataframe(filtered=filtered, sample=sample)
            num_classes = len(df.columns[1:])
            
            for app_name in app_names:
                i += 1
                print()
                print(f"Model: {i} of {num_models}")
                print("Filtered dataset:", filtered)
                print("Methodology:", methodology)
       
                model = build_model(app_name, activation=activation, num_classes=num_classes)

                img_size = model.input_shape[1:3]
                
                datasets = get_datasets(img_size, df=df)
                train_ds, test_ds, val_ds = configure_datasets_for_performance(datasets)

                model.name = model.name + "_" + ("filtered" if filtered else "unfiltered") + "_" + methodology
                print("Save file name:", model.name)

                history_clf = train_classifier(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=False)
                history_ft = train_fine_tuning(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=True)
                               
                history_df.to_csv(
                        str(histories_dir) + "/" + save_filename + ".csv",
                        index=False)              

                keras.utils.clear_session(free_memory=True)       

In [17]:
def compile_model(model, loss=None, metrics=[], fine_tuning=False):
    BASE_LR = 0.01
    lr = BASE_LR if not fine_tuning else BASE_LR / 10
    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        metrics=metrics
    )
    print_model_info(model)

In [18]:
def train_classifier(model=None, train_ds=None, val_ds=None, epochs=100, loss=None, metrics=[], fine_tuning=False):
    compile_model(model, loss=loss, metrics=metrics, fine_tuning=fine_tuning)
   
    cbs = get_callbacks(model, fine_tuning=False)
    
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=cbs)
    history_df = pd.DataFrame(history.history)
    history_df['model'] = model.name
    history_df['epoch'] = history.epoch
    history_df['type'] = "CLF"
    return history_df
    

In [19]:
def train_fine_tuning(model=None, train_ds=None, val_ds=None, epochs=100, callbacks=[], loss=None, metrics=[], fine_tuning=True):
    compile_model(model, loss=loss, metrics=metrics, fine_tuning=fine_tuning)
    
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=cbs)
    history_df = pd.DataFrame(history.history)
    history_df['model'] = model.name
    history_df['epoch'] = history.epoch
    history_df['type'] = "FT"
    return history_df
    

In [20]:
model = run_all(epochs=5, sample=False)


df shape: (5156, 28)

Model: 1 of 3
Filtered dataset: False
Methodology: multiclass
train size: 4124
val size: 516
test size: 516
Save file name: vgg16_unfiltered_multiclass
Model name: vgg16_unfiltered_multiclass
Input shape: (None, 224, 224, 3)
Optimizer name: adam learning_rate: 0.01
Loss: categorical_crossentropy
Metrics:
categorical_accuracy
{'name': 'f1_score_weighted', 'dtype': 'float32', 'average': 'weighted', 'threshold': None}
{'name': 'f1_score_per_class', 'dtype': 'float32', 'average': None, 'threshold': None}
Classifier layer activation function: softmax

Epoch 1/5


I0000 00:00:1744065419.290913   70807 service.cc:152] XLA service 0x7fccfc012800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744065419.291089   70807 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-07 23:36:59.818069: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744065420.517431   70807 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-07 23:37:01.415350: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_694', 152 bytes spill stores, 152 bytes spill loads

2025-04-07 23:37:01.630506: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusi

  2/258 ━━━━━━━━━━━━━━━━━━━━ 23s 90ms/step - categorical_accuracy: 0.0156 - f1_score_per_class: 0.0025 - f1_score_weighted: 0.0040 - loss: 5.6981             

I0000 00:00:1744065427.493880   70807 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


257/258 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - categorical_accuracy: 0.1889 - f1_score_per_class: 0.0143 - f1_score_weighted: 0.0747 - loss: 12.0734

2025-04-07 23:37:24.153136: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_694', 424 bytes spill stores, 424 bytes spill loads

2025-04-07 23:37:24.985502: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_694', 152 bytes spill stores, 152 bytes spill loads



258/258 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - categorical_accuracy: 0.1889 - f1_score_per_class: 0.0143 - f1_score_weighted: 0.0747 - loss: 12.0770

2025-04-07 23:37:31.782953: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_324', 8 bytes spill stores, 8 bytes spill loads

2025-04-07 23:37:32.499298: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_331', 124 bytes spill stores, 124 bytes spill loads

2025-04-07 23:37:32.714200: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_338', 608 bytes spill stores, 608 bytes spill loads

2025-04-07 23:37:32.793613: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_338', 412 bytes spill stores, 396 bytes spill loads

2025-04-07 23:37:37.058226: I external/l


Epoch 1: val_loss improved from inf to 12.99444, saving model to ../saved_models/<Functional name=vgg16_unfiltered_multiclass, built=True>.keras
258/258 ━━━━━━━━━━━━━━━━━━━━ 43s 128ms/step - categorical_accuracy: 0.1890 - f1_score_per_class: 0.0143 - f1_score_weighted: 0.0747 - loss: 12.0805 - val_categorical_accuracy: 0.2209 - val_f1_score_per_class: 0.0141 - val_f1_score_weighted: 0.0856 - val_loss: 12.9944
Epoch 2/5
257/258 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - categorical_accuracy: 0.2184 - f1_score_per_class: 0.0135 - f1_score_weighted: 0.0785 - loss: 13.0905
Epoch 2: val_loss did not improve from 12.99444
258/258 ━━━━━━━━━━━━━━━━━━━━ 15s 57ms/step - categorical_accuracy: 0.2183 - f1_score_per_class: 0.0135 - f1_score_weighted: 0.0785 - loss: 13.0916 - val_categorical_accuracy: 0.2209 - val_f1_score_per_class: 0.0141 - val_f1_score_weighted: 0.0856 - val_loss: 12.9944
Epoch 3/5
257/258 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - categorical_accuracy: 0.2184 - f1_score_per_class: 0.0135 - f

NameError: name 'train' is not defined